In [1]:
import pandas as pd,numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import mysql.connector
# from multiprocessing import Pool

exec(open('miniatlas_clusters.py').read())

flist=glob('binc.*tsv.gz')
binsize=128
mincov=5

def smooth_bins(df,smooth,binsize=binsize):
    if smooth==0:
        return df.set_index(['chr','pos'])
    dfs = []
    df_out=None
    for s in np.arange(-smooth//2, smooth//2):
        df1=df.copy()
        df1['pos']=df1['pos']+s*binsize
        df1=df1.set_index(['chr','pos'])
        if df_out is None:
            df_out=df1
        else:
            a,b=df_out.align(df1,join='outer',fill_value=0)
            df_out = a+b
    df_out=df_out
    return df_out

def get_table(chrom,clust,ctxt,mydb,binsize=binsize):
    chrom=chrom.replace('chr','')
    if ctxt=='CGN':
        track=f'mc_Round2_{clust}_{ctxt}_Merge_mc_{chrom}'
    else:
        track=f'mc_Round2_{clust}_{ctxt}_mc_{chrom}'
    query=f'SELECT {binsize}*FLOOR(position/{binsize}) as pos,sum(mc) as mc_sum,sum(h) as h_sum FROM {track} GROUP BY pos'
    df=pd.read_sql(query, mydb)
    df['chr']=chrom
    return df

from tqdm import tqdm
def clust2bigwig(clust,clust_label,ctxt,binsize=128,smooth=8,mincov=mincov,forceload=False):
    clust_label=clust_label.replace('/','-').replace(' ','_')
    file_bedgraph = f'/scratch/bedgraph/{clust}_{clust_label}_{ctxt}_sm{smooth}.bw'

    file_bigwig = file_bedgraph.split('/')[-1].replace('.bedgraph','.bw')
    file_bigwig = './bigwig/' + file_bigwig
    if os.path.exists(file_bigwig) and not forceload:
        print(f'File exists, skipping {file_bigwig}')
    else:
        os.system(f'touch {file_bigwig}')
        print(f'Processing {file_bigwig} sm={smooth} ctxt={ctxt}')
        dfs=[]
        chroms=[f'chr{i+1}' for i in np.arange(19)]+['chrX']
        with mysql.connector.connect(host="brainome",
            user="cndd_annoj",
            password="jonna_ddnc",
            database='miniatlas') as mydb:
            for chrom in tqdm(chroms):
                dfs.append(get_table(chrom,clust,ctxt,mydb,binsize=binsize))
        df=dfs[0].append(dfs[1:])
        df_sm = smooth_bins(df,smooth=smooth,binsize=binsize)

        df_bw = df_sm[f'mc_sum']/df_sm[f'h_sum']
        df_bw = df_bw[df_sm[f'h_sum']>mincov]
        df_bw.name = 'mC'
        df_bw=df_bw.reset_index()
        df_bw['end']=df_bw['pos']+binsize
        with open(file_bedgraph,'w') as f:
            f.write('track\ttype=bedGraph\n')
        df_bw[['chr','pos','end','mC']].to_csv(file_bedgraph,sep='\t',mode='a',header=None,index=False)

        cmd=f'bedGraphToBigWig {file_bedgraph} mm10_X.genome {file_bigwig}'
        print(cmd)
        os.system(cmd)
        print(f'-- Finished {file_bigwig}')

for cluster,label in zip(clusters,annots):
    for ctxt in ['CGN','CAC']:
        if ctxt=='CGN':
            clustuse=f'C{cluster}'
        else:
            clustuse=cluster
        df=clust2bigwig(clustuse,label,ctxt,binsize=128,smooth=8,mincov=mincov)
